In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import curve_fit
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler

df_dev=pd.read_csv('C:\\Users\\picca\\Documents\\DataScienceLab\\DataScience\\January_project\\dataset_wp\\DSL_Winter_Project_2024\\development.csv',sep= ',')
df_eval = pd.read_csv('C:\\Users\\picca\\Documents\\DataScienceLab\\DataScience\\January_project\\dataset_wp\\DSL_Winter_Project_2024\\evaluation.csv',sep= ',')

df_dev_coord = df_dev[['x', 'y']]
df_dev_train = df_dev.drop(columns=['x', 'y'])
fig, axs = plt.subplots(nrows=3, ncols=6, figsize=(24, 16))

def objective(x, mu, sigma, a, b, c) :
   return a*1/(np.sqrt(2*np.pi)*sigma*b*x)*np.exp(-((np.log(b*x)- mu)**2)/(2*sigma**2)) + c

listmafev = [280, 280, 280, 300,280, 300,280, 280,280, 280, 280, 300]
k=0
l=0
j=0
for i in [1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 13, 14]:
    print(i , "\n")
    df_dev_train_i = (df_dev[[f"pmax[{i}]"]])
    histog = axs[j % 3, l % 6].hist(df_dev_train_i, 300)
    print(listmafev[k])
    print(k)
    params, covariance = curve_fit(objective, list(histog[1])[:300], list(histog[0]), maxfev= listmafev[k])
    print(params)
    mu, sigma, a, b, c = params
    x = np.linspace(histog[1].min(), histog[1].max(), 300)
    y = objective(x,  mu, sigma, a, b, c)
    axs[j % 3, l % 6].plot(x, y, color='red', linestyle='solid', linewidth=2, label='Fit (Lognormal)')
    axs[j % 3, l % 6].set_title(f'pmax[{i}]')
    axs[j % 3, l % 6].legend(fontsize='medium')
    k += 1
    l += 1
    if(l % 6 == 0):
        j += 1

def normal(x, mu, sigma, a):
    return a*np.exp(-(x-mu)**2/(2*sigma**2))

listmafev = [ 280,280, 280]
k=0
l=0
for i in [0, 7, 12]:
    print(i , "\n")
    df_dev_train_i = df_dev[[f"pmax[{i}]"]]
    q999 = df_dev_train_i[f"pmax[{i}]"].quantile(.980)
    q001 = df_dev_train_i[f"pmax[{i}]"].quantile(.001)
    print(q999, q001)
    mask = (df_dev_train_i[f"pmax[{i}]"] < q999) & (df_dev_train_i[f"pmax[{i}]"] > q001)
    df_dev_train_i = df_dev_train_i.loc[mask]
    print(df_dev_train_i.shape)
    histog = axs[2, l].hist(df_dev_train_i, 200)
    print(listmafev[k])
    print(k)
    params, covariance = curve_fit(normal, list(histog[1])[:200], list(histog[0]), maxfev= listmafev[k])
    print(params)
    mu, sigma, a = params
    x = np.linspace(df_dev_train_i.min(), df_dev_train_i.max(), 200)
    y = normal(x, mu, sigma, a)
    axs[2, l].plot(x, y, color='red', linestyle='solid', linewidth=2, label='Fit (Normal)')
    axs[2, l].set_title(f'pmax[{i}]')
    axs[2, l].legend(fontsize='medium')
    l += 1
    k += 1


k=0
l=3
for i in [ 15, 16, 17]:
    print(i , "\n")
    df_dev_train_i = df_dev[[f"pmax[{i}]"]]
    q999 = df_dev_train_i[f"pmax[{i}]"].quantile(.995)
    q001 = df_dev_train_i[f"pmax[{i}]"].quantile(.005)
    print(q999, q001)
    mask = (df_dev_train_i[f"pmax[{i}]"] < q999) & (df_dev_train_i[f"pmax[{i}]"] > q001)
    df_dev_train_i = df_dev_train_i.loc[mask]
    print(df_dev_train_i.shape, df_dev_train_i.mean().shape, df_dev_train_i.std().shape)

    histog = axs[2, l].hist(df_dev_train_i, 100)
    axs[2, l].set_title(f'pmax[{i}]')
    l +=1
plt.savefig("pmax_distributions.png", dpi=300)  
plt.show()


In [ ]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score,mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression

features_eliminated = []

df_dev_drop_train = df_dev_train.drop(columns = features_eliminated)

fetures_names = df_dev_drop_train.columns
X = df_dev_drop_train.values
X.shape
y = df_dev_coord.values
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, shuffle=True, random_state=42, test_size = 0.20)


Creating list_euclideans_rf and list_depths_rf for max_depth hyperparameter tuning

In [ ]:
from sklearn.model_selection import ParameterGrid
import matplotlib.pyplot as plt
params = {
    "max_depth": [10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40],
    "n_estimators": [100],
    "random_state": [42],
    "max_features": ["sqrt"],
    "criterion": ["friedman_mse"]
}
list_euclideans_rf = []
list_depths_rf  = []
for config in ParameterGrid(params):
    print("\n" + "\n")
    regr_multi = MultiOutputRegressor(
    RandomForestRegressor(**config)
    )
    regr_multi.fit(X_train, y_train)
    y_pred = regr_multi.predict(X_valid)
    eucl_dist = np.sum(np.sum(((y_valid - y_pred) )**(2), axis=1)**(1/2), axis=0) * 1/(y_valid.size / 2)
    list_euclideans_rf = list_euclideans_rf + [eucl_dist]
    list_depths_rf = list_depths_rf + [config.get("n_estimators")]
    print(config, "\t ---> " ,eucl_dist)

params = {
    "max_depth": [16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40],
    "n_estimators": [100],
    "random_state": [42],
    "max_features": ["sqrt"],
    "criterion": ["friedman_mse"]
}
list_euclideans_et = []
list_depths_et = []
for config in ParameterGrid(params):
    print("\n" + "\n")
    regr_multi = MultiOutputRegressor(
    ExtraTreesRegressor(**config)
    )
    regr_multi.fit(X_train, y_train)
    y_pred = regr_multi.predict(X_valid)
    eucl_dist = np.sum(np.sum(((y_valid - y_pred) )**(2), axis=1)**(1/2), axis=0) * 1/(y_valid.size / 2)
    list_euclideans_et = list_euclideans_et + [eucl_dist]
    list_depths_et = list_depths_et + [config.get("n_estimators")]
    print(config, "\t ---> " ,eucl_dist)

list_euclideans_dt = []
list_depths_dt = []
for config in ParameterGrid(params):
    print("\n" + "\n")
    regr_multi = MultiOutputRegressor(
    DecisionTreeRegressor(**config)
    )
    regr_multi.fit(X_train, y_train)
    y_pred = regr_multi.predict(X_valid)
    eucl_dist = np.sum(np.sum(((y_valid - y_pred) )**(2), axis=1)**(1/2), axis=0) * 1/(y_valid.size / 2)
    list_euclideans_dt = list_euclideans_dt + [eucl_dist]
    list_depths_dt = list_depths_dt + [config.get("max_depth")]
    print(config, "\t ---> " ,eucl_dist)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
list_depths = [10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40]
list_depths_et = [16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40]
ax.plot(list_depths, list_euclideans_rf, c='red', linestyle ="--", label= "random forest", linewidth = 3)
ax.plot(list_depths, list_euclideans_dt, c='blue', label="decision tree",  linewidth = 3)
ax.plot(list_depths_et, list_euclideans_et, c='green', linestyle='dotted', label="extra trees", linewidth = 3)
ax.legend(loc="upper right",  fontsize="20" )
plt.xlabel("Max depth")
plt.ylabel("Avg. Eucl. ditance")
plt.savefig("tuning_maxdepth.pdf", dpi=300)
plt.show()

Creating list_euclideans_rf and list_depths_rf for n_estimators hyperparameter tuning

In [ ]:
from sklearn.model_selection import ParameterGrid
import matplotlib.pyplot as plt
params = {
    "max_depth": [18],
    "n_estimators": [50, 100, 150, 200,  250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800],
    "random_state": [42],
    "max_features": ["sqrt"],
    "criterion": ["friedman_mse"]
}
list_euclideans_rf = []
list_depths_rf  = []
for config in ParameterGrid(params):
    print("\n" + "\n")
    regr_multi = MultiOutputRegressor(
    RandomForestRegressor(**config)
    )
    regr_multi.fit(X_train, y_train)
    y_pred = regr_multi.predict(X_valid)
    eucl_dist = np.sum(np.sum(((y_valid - y_pred) )**(2), axis=1)**(1/2), axis=0) * 1/(y_valid.size / 2)
    list_euclideans_rf = list_euclideans_rf + [eucl_dist]
    list_depths_rf = list_depths_rf + [config.get("n_estimators")]
    print(config, "\t ---> " ,eucl_dist)

params = {
    "max_depth": [18],
    "n_estimators": [50, 100, 150, 200,  250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800],
    "random_state": [42],
    "max_features": ["sqrt"],
    "criterion": ["friedman_mse"]
}
list_euclideans_et = []
list_depths_et = []
for config in ParameterGrid(params):
    print("\n" + "\n")
    regr_multi = MultiOutputRegressor(
    ExtraTreesRegressor(**config)
    )
    regr_multi.fit(X_train, y_train)
    y_pred = regr_multi.predict(X_valid)
    eucl_dist = np.sum(np.sum(((y_valid - y_pred) )**(2), axis=1)**(1/2), axis=0) * 1/(y_valid.size / 2)
    list_euclideans_et = list_euclideans_et + [eucl_dist]
    list_depths_et = list_depths_et + [config.get("n_estimators")]
    print(config, "\t ---> " ,eucl_dist)
   

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
list_nestimators = [50, 100, 150, 200,  250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800]
ax.plot(list_nestimators, list_euclideans_rf, c='red', linestyle ="--", label= "random forest", linewidth = 3)
ax.plot(list_nestimators, list_euclideans_dt, c='green', linestyle ="dotted", label="extra trees",  linewidth = 3)
ax.legend(loc="upper right",  fontsize="20" )
plt.xlabel("N. Estimators")
plt.ylabel("Avg. Eucl. ditance")
plt.savefig("tuning_nestimators.pdf", dpi=300)
plt.show()